In [1]:
function read_output(filename)
    content = []
    f = open(filename)
    
    for line in readlines(f)
        line_content = String(line)
        push!(content, line_content)
    end
    return content
end

read_output (generic function with 1 method)

In [2]:
# Day 7 Part 1
filename = "data/day7_data.txt"
# filename = "data/day7_test.txt"

terminal_output = read_output(filename)

# struct FileTree  # make your own trees
#     data::String
#     children::Vector{FileTree}
# end;
# file_tree = FileTree("dir /", [])

pwd = []
ls_on = []
is_unknown = false
dirnames = Dict{String, Int64}()
for line in terminal_output
    if occursin("\$ cd /" ,line)
        pwd = ["/"]
        dirnames["/"] = 0
    elseif occursin("\$ cd ..", line)
        pop!(pwd)
    elseif line[1:4] == "\$ cd"
        dir = line[6:length(line)]
        push!(pwd, dir)
        if !(dir in keys(dirnames))
            dirnames[dir] = 0
        end
    end

    if !occursin("\$", line)
        regex_file = r"([0-9]+) .+"
        regex_match = match(regex_file, line)
        if regex_match !== nothing 
            for dir in pwd
                dirnames[dir] = dirnames[dir] + parse(Int64, regex_match.captures[1])
                # println(dir, " + ", parse(Int64, regex_match.captures[1]), " = ", dirnames[dir])     

            end
        end
    end
    if occursin("\$ ls", line)
        push!(ls_on, pwd)
    end
end

if length(ls_on) > length(Set(ls_on))
    println("duplicate")
end

sum = 0
for (dir, size) in dirnames
    # print(rpad.(dir,15, " "), rpad.(string(size), 10, " "))
    if size <= 100000
        sum = sum + size
        # print("<--")
    end
    # println()
end
println("Sum: ", sum)


Sum: 1036344
1117929


duplicate
Sum: 1036344


LoadError: UndefVarError: Sum not defined

In [3]:
function print_folder(folder::Dict, level::Int)
    indentation = ""
    for i in 1:level
        indentation *= "  "
    end 
    for (key, value) in folder
        print(indentation, key)
        if value isa Dict
            println()
            print_folder(value, level+1)
        else 
            println(", ", value)
        end
    end
end

print_folder (generic function with 1 method)

In [4]:
function get_folder_size(folder::Dict, folder_sizes::Dict{String, Int}, pwd)
    size = 0
    for (key, value) in folder
        if value isa Dict
            push!(pwd, key)
            folder_path = join(pwd, "/")
            folder_sizes[folder_path] = get_folder_size(value, folder_sizes, pwd)
            size += folder_sizes[folder_path]
        else 
            size += value
        end
    end
    pop!(pwd)
    return size
end

get_folder_size (generic function with 1 method)

In [5]:
function get_content(folder, terminal_output, row)
    # println("Length ", length(terminal_output))
    while row < length(terminal_output)
        if occursin("\$ ls", terminal_output[row])
            j = row+1
            while j <= length(terminal_output) && !occursin("\$", terminal_output[j])
                regexp = r"(.+) (.+)" 
                match_res = match(regexp, terminal_output[j])
                type = string(match_res.captures[1])
                name = string(match_res.captures[2])
                if !(name in keys(folder))
                    if type == "dir"
                        folder[name] = Dict()
                    else
                        size = parse(Int64, type)
                        folder[name] = size
                    end
                end
                j += 1
            end
            row = j
        elseif occursin("\$ cd ..", terminal_output[row]) 
            row += 1
            return folder, row
        elseif occursin("\$ cd", terminal_output[row])    
            name = terminal_output[row][6:length(terminal_output[row])]

            folder[name], row = get_content(folder[name], terminal_output, row+1)    
        end
    # print_folder(folder, 0)
    end
    return folder, row
end

get_content (generic function with 1 method)

In [6]:
# Day 7 Part 1 Nested dict, recursion
filename = "data/day7_data.txt"
# filename = "data/day7_bjorn.txt"
# filename = "data/day7_test.txt"
terminal_output = read_output(filename)

struct Item
    size::Int64
    content
end

root_folder = Dict("/" => Dict())
pwd = ["/"]
root_folder["/"], row = get_content(root_folder["/"], terminal_output[2:length(terminal_output)], 1)
print_folder(root_folder, 0)
folder_sizes = Dict{String, Int}()
size = get_folder_size(root_folder, folder_sizes, ["/"])
sum = 0
for (key, value) in folder_sizes
    print(key, ":", value)
    if value <= 100000
        print(" *")
        sum += value
    end
    println()
end
println("Sum: ", sum)
wrong = [639542, 1297159]
bjorn = 1447046
if sum == bjorn
    println("BJÖRN!!!")
end

if sum in wrong
    println("WRONG!!!!!")
end

/
  dwhl.vvb, 124868
  dwhl.nrn, 240038
  qjphltd
    tvplqllb
      ftwz.fzj, 84875
      hpfr
        mfprqmdp.pnl, 250910
        wlfprc
          mfprqmdp.pnl, 143769
        fdpg, 131524
        qpl.qtm, 174858
        jvddlfsf
          zpcc.wzt, 78145
        sjncl.cjb, 271840
        dwhl
          gtmgtq, 124522
        bbgbnr
          crgzcmrt.jlr, 21309
          chvtw, 139093
          nsjfgmrs
            tvsqvsrf.plp, 167856
            vztfzgrd
              fwjrgdbb, 213100
            ppwm
              ssl.fnd, 161664
          qqt.mvb, 22977
          srdgnb.zbj, 129496
      wdgzsg
        wlfprc
          rvgrjsps
            wrlzz.nwn, 62849
          wlfprc.ghn, 210367
          cjbjffhs.rlc, 180812
          dwhl
            sbrdjv.cdt, 220631
            swjgs.glp, 24851
          vql.lww, 229144
        jzhvmj
          sfrlrc.jtj, 249647
          vcn, 276429
          dnqs, 2388
          wlfprc
            mfprqmdp.pnl, 179811
        wfmttzc.cdb, 204706
 

In [19]:
# Day 7 Part 2
filename = "data/day7_data.txt"
# filename = "data/day7_bjorn.txt"
# filename = "data/day7_test.txt"
terminal_output = read_output(filename)

root_folder = Dict("/" => Dict())
pwd = ["/"]
root_folder["/"], row = get_content(root_folder["/"], terminal_output[2:length(terminal_output)], 1)
# print_folder(root_folder, 0)
folder_sizes = Dict{String, Int}()
size = get_folder_size(root_folder, folder_sizes, ["/"])
dir_to_delete = "..."
size_to_be_freed = size
tot_disk_size = 70000000
unused_space = tot_disk_size - size
needed_space = 30000000 - unused_space
println("Used Space: ", size)
println("Unused space: ", unused_space)
println("Needed space: ", needed_space)
for (key, value) in folder_sizes
    print(string(last(split(key, "/"))), ":", value)

    if value >= needed_space
        print(" *")
        if  value < size_to_be_freed
            print(" *")
            size_to_be_freed = value
            dir_to_delete = string(last(split(key, "/")))
        end
    end
    println()
end

println("Folder to delete: ", dir_to_delete)
println("Size to be freed: ", size_to_be_freed)

wrong = [3866390]
bjorn = 578710
if size_to_be_freed == bjorn
    println("BJÖRN!!!")
end
if size_to_be_freed in wrong
    println("WRONG!!!!!")
end

Used Space: 43636666
Unused space: 26363334
Needed space: 3636666
qbpvrr:1497094
vhpqjr:194144
bgwcrp:114175
cvm:379765
cpf:280711
qbpvrr:545286
pjm:147674
mfpcdbg:2675259
dqf:548836
mlhwcg:115287
shrpdcg:665316
tchbjclg:1238189
wpslpf:87578
blhrls:110313
:43636666 *
pcfr:660154
vvwb:340821
dvrb:216103
ntvmrrd:46209
tchbjclg:113952
gzfqggrr:162528
gvrfbt:200067
tchbjclg:227846
ztrz:187968
sbzf:257770
vbmqtf:151429
zhw:114175
nvgrzrbr:590540
cfcfnspz:1845406
qbpvrr:22955
gdlqtlv:162602
dtbws:210127
jhj:172487
mjjj:301212
gqv:229300
qtdh:215668
fzjdz:923097
bddslhrg:242433
mjjj:111397
gpbhtj:1472261
scsp:30654
hfns:3487907
gwz:13023723 * *
jgrmrzlh:668042
bbjnrhf:2363840
wdvqhn:1432500
lpbc:188528
ptfsh:2229740
tchbjclg:2595684
mpqtgd:99833
vpn:286479
bddslhrg:78175
vlnbm:143113
mmzcqr:162528
tbdff:226438
mdvqv:414795
nnpvwtvt:1319760
bhvzrsv:1383343
bddslhrg:81585
ghwnd:354223
mrlvp:197237
cmwwg:926000
hfg:1680
nmcp:273633
nzpcvjhp:352746
mjjj:197237
rgrcfr:87578
hmd:1451431
nfdv:557360